In [11]:
import numpy as np 
import pandas as pd
from sklearn import *
import lightgbm as lgb
from catboost import Pool,CatBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

pd.set_option("display.precision", 8)

from functools import partial
import scipy as sp

import os
from sklearn.metrics import f1_score

import pywt 
from statsmodels.robust import mad

import scipy
from scipy import signal
from scipy.signal import butter, deconvolve, find_peaks, peak_widths, peak_prominences

from numpy.fft import *

import time
import math
from numba import jit
from math import log, floor
from sklearn.neighbors import KDTree
 

train = pd.read_csv('/Users/siero5335/channel/train2.csv')
test = pd.read_csv('/Users/siero5335/channel/test2.csv')

sample_submission = pd.read_csv('/Users/siero5335/channel/sample_submission.csv')


In [15]:
def remove_false_peak(signal, p1, p2, maxDistance=10):
    peak_diff = np.diff(p2)
    if len(peak_diff) == 0:
        return p1
    ticks = []
    for i, d in enumerate(peak_diff):
        ratio = signal[p2[i+1]]/signal[p2[i]]
        if d < maxDistance and -0.25 > ratio and ratio > -4:
            ticks.append((p2[i], p2[i+1]))
    mask = np.array([True]*len(p1))
    for i, j in ticks:
        mask = mask &amp; ((p1 < i) | (p1 > 500+j))
    return p1[mask]


def get_peaks(signal):
    p1_1, _ = find_peaks(signal, height=[5, 100])
    p1_2, _ = find_peaks(-signal, height=[5, 100])
    p1 = np.union1d(p1_1, p1_2)
    n_peaks, _ = find_peaks(-signal, height=[10, 100])
    p_peaks, _ = find_peaks(signal, height=[10, 100])
    p2 = np.union1d(n_peaks, p_peaks)
    p = remove_false_peak(signal, p1, p2, maxDistance=10)
    return np.intersect1d(p1_1, p), np.intersect1d(p1_2, p)


def extract_peak_feature(signal):
    p_peaks, n_peaks = get_peaks(signal)
    
    num_p, num_n = len(p_peaks), len(n_peaks)

    sig_peak_width = np.concatenate(
        [peak_widths(signal, p_peaks)[0], peak_widths(-signal, n_peaks)[0]])
    sig_peak_height = abs(signal[np.concatenate([p_peaks, n_peaks])])

    if num_n or num_p:
        height_mean = sig_peak_height.mean()

        width_mean = sig_peak_width.mean()

        return np.array([height_mean, width_mean])
    else:
        return np.zeros(2)

In [16]:
window_sizes = [100, 500, 1000, 2500, 5000]

In [17]:
for window in window_sizes:
    train["rolling_height_mean_chris_" + str(window)] = train['signal_chris'].rolling(window=window).apply(lambda x: extract_peak_feature(x)[0])
    train["rolling_width_mean_chris_" + str(window)] = train['signal_chris'].rolling(window=window).apply(lambda x: extract_peak_feature(x)[1])

train.to_csv('/Users/siero5335/channel/train_rolling_signal_chris.csv')

/Users/siero5335/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning

/Users/siero5335/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning



In [18]:
for window in window_sizes:
    test["rolling_height_mean_chris_" + str(window)] = test['signal_chris'].rolling(window=window).apply(lambda x: extract_peak_feature(x)[0])
    test["rolling_width_mean_chris_" + str(window)] = test['signal_chris'].rolling(window=window).apply(lambda x: extract_peak_feature(x)[1])
    
test.to_csv('/Users/siero5335/channel/test_rolling_signal_chris.csv')

/Users/siero5335/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning

/Users/siero5335/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning

